<a href="https://colab.research.google.com/github/kyugorithm/TIL/blob/main/study_CAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

코랩 장시간 세션 유지 (https://naenjun.tistory.com/18)

In [ ]:
import os
from google.colab import drive 
!nvidia-smi
drive.mount('/content/gdrive/')

작업경로 pytorch-CycleGAN-Unet-D 로 변경

In [ ]:
os.chdir('/content/gdrive/MyDrive/')
root_dir = os.getcwd()
print(root_dir)

In [ ]:
import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.nn as nn

In [ ]:
transform = transforms.Compose([
                                transforms.Resize(224),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))

])

In [ ]:
trainset    = torchvision.datasets.STL10(root = './dataset', split = 'train', transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 64, shuffle = True)
testset    = torchvision.datasets.STL10(root = './dataset', split = 'test', transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = 64, shuffle = True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
classes = ('airplance', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck')

def imshow(img):
  img = 0.5 * img + 0.5
  npimg = img.numpy()
  plt.imshow(np.transpose(npimg,(1,2,0)))

dataiter = iter(trainloader)
images, labels = dataiter.next()
imshow(torchvision.utils.make_grid(images))
# print(' '.join('%5s' % classes[labels[i] for i in range(4)]))

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Sequential(
            #3 224 128
            nn.Conv2d(3, 64, 3, padding=1),nn.LeakyReLU(0.2),nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, 3, padding=1),nn.LeakyReLU(0.2),nn.BatchNorm2d(64),
            nn.MaxPool2d(2, 2),
            #64 112 64
            nn.Conv2d(64, 128, 3, padding=1),nn.LeakyReLU(0.2),nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, 3, padding=1),nn.LeakyReLU(0.2),nn.BatchNorm2d(128),
            nn.MaxPool2d(2, 2),
            #128 56 32
            nn.Conv2d(128, 256, 3, padding=1),nn.LeakyReLU(0.2),nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, 3, padding=1),nn.LeakyReLU(0.2),nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, 3, padding=1),nn.LeakyReLU(0.2),nn.BatchNorm2d(256),
            nn.MaxPool2d(2, 2),
            #256 28 16
            nn.Conv2d(256, 512, 3, padding=1),nn.LeakyReLU(0.2),nn.BatchNorm2d(512),
            nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),nn.BatchNorm2d(512),
            nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),nn.BatchNorm2d(512),
            nn.MaxPool2d(2, 2),
            #512 14 8
            nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),nn.BatchNorm2d(512),
            nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),nn.BatchNorm2d(512),
            nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),nn.BatchNorm2d(512),
            nn.MaxPool2d(2, 2)
        )
        #512 7 4

        self.avg_pool = nn.AvgPool2d(7)
        #512 1 1
        self.classifier = nn.Linear(512, 10)
        """
        self.fc1 = nn.Linear(512*2*2,4096)
        self.fc2 = nn.Linear(4096,4096)
        self.fc3 = nn.Linear(4096,10)
        """

    def forward(self, x):

        #print(x.size())
        features = self.conv(x)
        #print(features.size())
        x = self.avg_pool(features)
        #print(avg_pool.size())
        x = x.view(features.size(0), -1)
        #print(flatten.size())
        x = self.classifier(x)
        #x = self.softmax(x)
        return x, features

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")




net = Net()
net = net.to(device)
param = list(net.parameters())
print(len(param))
for i in param:
    print(i.shape)
#print(param[0].shape)

In [ ]:
classes =  ('airplance', 'bird', 'car', 'cat', 'deer', 'dog', 'horse', 'monkey', 'ship', 'truck')
import torch.optim as optim

criterion = nn.CrossEntropyLoss().cuda()
optimizer = optim.Adam(net.parameters(),lr=0.0001)


In [ ]:
for epoch in range(100):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        #print(inputs.shape)
        # forward + backward + optimize
        outputs,f = net(inputs)
        #print(outputs.shape)
        #print(labels.shape)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print('[epoch:%d] loss: %.3f' % (epoch + 1, running_loss / (i+1)))
    running_loss = 0.0

print('Finished Training')

class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        outputs,_ = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.cuda()
        labels = labels.cuda()
        outputs, f = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))